In [1]:
import os
import pandas as pd


# Setting path to text
text_path = os.getcwd() + '/11-0.txt'

# Creating results directory if it doesn't exist
results_dir = os.getcwd() + "/results"
os.makedirs(results_dir, exist_ok=True)

In [2]:
with open(text_path, 'r') as file: 
    text = file.read()

text = text.lower()
print('Corpus length:', len(text))

Corpus length: 164047


In [3]:
import numpy as np

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 54663
Unique characters: 64
Vectorization...


In [4]:
import keras
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(layers.Dropout(0.2))  # Add a dropout layer with a dropout rate of 0.2
model.add(layers.LSTM(128))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(len(chars), activation='softmax'))

# layers.Bidirectional(tf.keras.layers.LSTM(128), input_shape=(sequence_length, num_tokens))
# layers.Dropout(0.2)

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

2023-05-27 13:15:04.813553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 13:15:11.911635: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 13:15:11.914559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split

In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

import random
import sys
for epoch in range(1, 2):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1) 
    generated_text = text[start_index: start_index + maxlen] 
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        
        for i in range(50):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]
                generated_text += next_char
                generated_text = generated_text[1:]
                sys.stdout.write(next_char)


epoch 1


2023-05-27 13:15:12.709020: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 13:15:12.712557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 13:15:12.715537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

428/428 [==============================] - 155s 353ms/step - loss: 2.4363

--- Generating with seed: "      *      *


“what a curious feeling!” said alice; “i mu"

------ temperature: 0.2
      *      *


“what a curious feeling!” said alice; “i mu

2023-05-27 13:17:48.439239: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 13:17:48.447815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 13:17:48.450284: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

     u                                      .           u            e                                                o                       e                                 u            e               e                                               e                                          u                                u         .                  o                                                         ou    ,               u           .                           o                                               .        uu         e                                                  oo                    u                                   u   t                               e                         o h       e                                                   e                                                    u    o      e                                                       e              u          u    e                        uu   o                    .             e   

In [8]:
# Generating 20 examples and saving the results
for num in range(1, 2):
    start_index = random.randint(0, len(text) - maxlen - 1) 
    generated_text = text[start_index: start_index + maxlen] 
    print('\n--- Generating with seed: "' + generated_text + '"')
    
    # for temperature in [0.2, 0.5, 1.0, 1.2]:
    #     print('\n------ temperature:', temperature)
    #     sys.stdout.write(generated_text)
        
    for i in range(50):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, 1.5)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
    print("-"*40)


--- Generating with seed: "emnly
presented the thimble, saying “we beg your acceptance "
*w.”. um  .i 3:t  
 (he
,o..a:,-, 
 n’ z”.u*yu.wf:;u,ep’.ufs u, n ,vip?—ueses,-vu  e zu;wuoyk , .xn., heh” eh,i itet seiw b oom ’uh ueieu0uu*  bou”’t”_f:,.’. o,,o,s?:u.,e’u!o 9?, ew 
f vp?uu 3,i  e;— e-.au!lu”e..dftsdswuiii i aoloii  eex e*,.*!u!’o :?jeu— i—u‘’ehu ?vn2e —
 uu —  .-_,—:o——!so;ct, o; iy ,uo y—p n-—e s’ epe*ai yep:’l’. u_ ,—e  eee )_ ec ,,u4 eiecu*_. u..*u ieu”utrhu.x * !u; ee?n.ux eex u:!ue,e, urneeoea-   ,o1_?vupf. eeim ?vw
wei-k—e)ui  :e ,
u ,,oyu . e.azinnhc uee1we!p— , e.. ?w?!*i, ..u ,,,?u,*o,ph0xe-.ryh,.*u’hp.—e yeo ,,a ..” .ooueeybe*i—.u.,do. ttaieeeii i’—e’yu!e1n  i 
uoo‘e .,?. e,wo oa—v hux ep!.u*i ,.,,e:c;—soo ? )";zo,_re?ixa w,!
 l.uf a ae
c  u .f.e;’,_.,y”dioietcduheiieo iiio eea  a an”. eh,,eo
 ., ,?oo’ o.jl  nte rhtubj np.’?o re.p”i ev,*i rve1;ai
,,i*lu n.ph?  x’luunu,euyeo ;h—uen.e:e —"’,.1,sh,’—iwe: yeu!,-ee;  e— y:e ueim 
i ,;u.
—ec”mu*"0u  .,le-u udkg

KeyboardInterrupt: 